In [1]:
d = '../data/pre/' # raw data directory

import os
import pandas as pd
import numpy as np
from collections import Counter
from time import time
from imblearn.under_sampling import RandomUnderSampler


# ## Load all csv files into pandas DataFrames
'''user ad position app_categories'''

user_installedapps = pd.read_csv('%suser_installedapps.csv' % d)
user_app_actions = pd.read_csv('%suser_app_actions.csv' % d)

In [2]:
tr_df = pd.read_csv('../data/pre/new_generated_train.csv', index_col=0)
te_df = pd.read_csv('../data/pre/new_generated_test.csv', index_col=0)

In [28]:
aC = pd.read_csv('../data/pre/app_categories.csv')

In [5]:
ui = user_installedapps.groupby('userID').apply(lambda df: tuple(df.appID.values)).reset_index()

In [32]:
aC.appID.max()

433269

In [6]:
ui.columns=['userID', 'insApps']

In [8]:
tr_df_ui = pd.merge(tr_df, ui, on='userID', how='left')

In [15]:
tr_df.columns[np.array(uni)<10]

Index(['label', 'connectionType', 'telecomsOperator', 'appPlatform', 'gender',
       'education', 'marriageStatus', 'haveBaby', 'sitesetID', 'positionType',
       'weekDay'],
      dtype='object')

In [21]:
uis.head()

userID
1     79
9      3
10    97
12    38
14    62
dtype: int64

In [34]:
user_app_actions.head()

,userID,installTime,appID
0,1,182032,933
1,5,141354,589
2,5,251727,565
3,5,260932,596
4,5,261318,536


In [35]:
uact_cate = pd.merge(user_app_actions, aC, on='appID', how='left')

In [36]:
uact_cate.head()

,userID,installTime,appID,appCategory
0,1,182032,933,106
1,5,141354,589,109
2,5,251727,565,203
3,5,260932,596,106
4,5,261318,536,104


In [47]:
uact_cate.installTime_d.max()

92

In [40]:
uact_cate['installTime_d'] = uact_cate.installTime.map(lambda x: int(str(x)[0:2]))

In [46]:
uact_cate = uact_cate.groupby('userID').apply(lambda df: tuple(df[['appID', 'appCategory', 'installTime_d']].values)).reset_index()

,userID,0
0,1,"([933, 106, 18],)"
1,5,"([589, 109, 14], [565, 203, 25], [596, 106, 26..."
2,12,"([589, 109, 61], [536, 104, 61], [1014, 0, 71]..."
3,14,"([49946, 0, 51], [59822, 0, 51], [78248, 0, 51..."
4,19,"([12129, 108, 21],)"
5,20,"([357, 201, 21], [587, 408, 21], [914, 503, 21..."
6,21,"([669, 209, 10], [897, 209, 10], [2678, 209, 1..."
7,25,"([71137, 405, 61], [379, 203, 12], [906, 0, 13..."
8,31,"([536, 104, 51],)"
9,38,"([359, 203, 28], [360, 201, 28], [361, 2, 28],..."


### user actions

In [ ]:
ua  = user_app_actions.groupby('userID').apply(lambda df: tuple(df.appID.values)).reset_index()
us.columns = ['userID', 'actApps']

In [17]:
tr_df.appID.describe()

count    3.749528e+06
mean     3.703212e+02
std      1.440570e+02
min      1.400000e+01
25%      3.600000e+02
50%      4.650000e+02
75%      4.650000e+02
max      4.720000e+02
Name: appID, dtype: float64

In [33]:
user_app_actions.head()

,userID,installTime,appID
0,1,182032,933
1,5,141354,589
2,5,251727,565
3,5,260932,596
4,5,261318,536


In [ ]:
ua_with_cate = pd.merge(ua)